In [1]:
import tensorflow as tf
import tflearn
import numpy as np

In [2]:
class LSTM:
    def __init__(self, vocabulary_size=200, num_nodes=128, batch_size=20, num_unrollings=2, name=""):
        self.num_nodes = num_nodes
        self.reuse = False
        self.vocabulary_size = vocabulary_size
        self.num_nodes = num_nodes
        self.num_unrollings = num_unrollings
        self.batch_size = batch_size
        self.name = name
        
        with tf.variable_scope("{}_LSTM_var".format(self.name) ,reuse=self.reuse):
            with tf.variable_scope("input_gate"):
                self.ix = tf.get_variable("ix", [vocabulary_size, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.im = tf.get_variable("im", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ib = tf.get_variable("ib", [1, num_nodes], 
                                      tf.float32, tf.constant_initializer(0))
            with tf.variable_scope("forget_gate"):
                self.fx = tf.get_variable("fx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.fm = tf.get_variable("fm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.fb = tf.get_variable("fb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("output_gate"):
                self.ox = tf.get_variable("ox", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.om = tf.get_variable("om", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.ob = tf.get_variable("ob", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
            
            with tf.variable_scope("memory_cell"):
                self.cx = tf.get_variable("cx", [vocabulary_size, num_nodes], tf.float32, 
                                     tf.truncated_normal_initializer(stddev=0.1))
                self.cm = tf.get_variable("cm", [num_nodes, num_nodes], 
                                      tf.float32, tf.truncated_normal_initializer(stddev=0.1))
                self.cb = tf.get_variable("cb", [1, num_nodes], 
                                     tf.float32, tf.constant_initializer(0))
                
            with tf.variable_scope("output_layer"):
                self.w = tf.get_variable("w", [num_nodes, vocabulary_size], tf.float32,
                                        tf.truncated_normal_initializer(stddev=0.1))
                self.b = tf.get_variable("b", [vocabulary_size], tf.float32,
                                        tf.constant_initializer(0))
            
            self.saved_output = tf.get_variable("saved_output", [batch_size, num_nodes], 
                                                tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                trainable=False)
            self.saved_state = tf.get_variable("saved_state", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False)
            '''self.saved_state_backward = tf.get_variable("saved_state_backward", [batch_size, num_nodes], 
                                                       tf.float32, tf.truncated_normal_initializer(stddev=0.1), 
                                                       trainable=False, name="backward_state")'''
                      
        self.variables = tf.get_collection(tf.GraphKeys.VARIABLES)
        
    def lstm_cell(self, inputs, hidden_layer, state): 
        
        with tf.variable_scope("{}_LSTM_cell".format(self.name), reuse=self.reuse):
            input_gate = tf.sigmoid(tf.matmul(inputs, self.ix) + tf.matmul(hidden_layer, self.im) + self.ib,
                                    name="input_gate")
            forget_gate = tf.sigmoid(tf.matmul(inputs, self.fx) + tf.matmul(hidden_layer, self.fm) + self.fb,
                                    name="forget_gate")
            update_gate = tf.tanh(tf.matmul(inputs, self.cx) + tf.matmul(hidden_layer, self.cm) + self.cb,
                            name="update_gate")
            state = tf.add(tf.mul(forget_gate, state), tf.mul(input_gate, update_gate),
                           name="state")
            output_gate = tf.sigmoid(tf.matmul(inputs, self.ox) + tf.matmul(hidden_layer, self.om) + self.ob,
                                     name="output_gate") 
            output = tf.mul(output_gate, tf.tanh(state), name="output")
        
        #Note that, the "output" is equal to the hidden state
        return output, state   
    
    def lstm_process(self, inputs, hidden_layer, state):
        train_data = list()
        for _ in range(self.num_unrollings + 1):
            train_data.append(
                tf.placeholder(tf.float32, shape=[self.batch_size, self.vocabulary_size]))
        train_inputs = train_data[:self.num_unrollings]
        train_labels = train_data[1:]
        
        outputs = list()
        output = self.saved_output
        state = self.saved_state
        for i in train_inputs:
            output, state = self.lstm_cell(i, output, state)
            outputs.append(output)

        '''with tf.control_dependencies([self.saved_output.assign(output),
                                     self.saved_state.assign(state)]):
            
            logits = tf.matmul(tf.concat(0, outputs), self.w) + self.b
            loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(tf.concat(0, logits), 
                                                        tf.concat(0, train_labels), name="loss"))
            
            global_step = tf.Variable(0)
            learning_rate = tf.train.exponential_decay(
                10.0, global_step, 5000, 0.1, staircase=True)
            optimizer = tf.train.RMSPropOptimizer(learning_rate)
            gradient, v = zip(*optimizer.compute_gradients(loss))
            
            
            tf.add_to_collection("loss", loss)
            tf.add_to_collection("logits", logits)
            tf.add_to_collection("train_labels", train_labels)
            
        '''
        #It seems we do not need those lines above
            
        return outputs
       
    def __call__(self, inputs):
        self.lstm_process(inputs)
        

In [ ]:
class attention:
    def __init__(self, h, s):
        pass

In [5]:
L = LSTM(name="encoder")

In [6]:
h = tf.Variable(tf.truncated_normal((1,128)))
state = tf.Variable(tf.truncated_normal((1,128)))
test_inputs = tf.placeholder(tf.float32, shape=[1, 200])

In [7]:
L.lstm_cell(test_inputs, h, state)

(<tf.Tensor 'encoder_LSTM_cell/output:0' shape=(1, 128) dtype=float32>,
 <tf.Tensor 'encoder_LSTM_cell/state:0' shape=(1, 128) dtype=float32>)

In [9]:
L.lstm_process(test_inputs, h, state)

[<tf.Tensor 'encoder_LSTM_cell_1/output:0' shape=(20, 128) dtype=float32>,
 <tf.Tensor 'encoder_LSTM_cell_2/output:0' shape=(20, 128) dtype=float32>]

In [10]:
for i in L.variables:
    print("{},{}".format(i.name, i.get_shape()))

is_training:0,()
encoder_LSTM_var/input_gate/ix:0,(200, 128)
encoder_LSTM_var/input_gate/im:0,(128, 128)
encoder_LSTM_var/input_gate/ib:0,(1, 128)
encoder_LSTM_var/forget_gate/fx:0,(200, 128)
encoder_LSTM_var/forget_gate/fm:0,(128, 128)
encoder_LSTM_var/forget_gate/fb:0,(1, 128)
encoder_LSTM_var/output_gate/ox:0,(200, 128)
encoder_LSTM_var/output_gate/om:0,(128, 128)
encoder_LSTM_var/output_gate/ob:0,(1, 128)
encoder_LSTM_var/memory_cell/cx:0,(200, 128)
encoder_LSTM_var/memory_cell/cm:0,(128, 128)
encoder_LSTM_var/memory_cell/cb:0,(1, 128)
encoder_LSTM_var/output_layer/w:0,(128, 200)
encoder_LSTM_var/output_layer/b:0,(200,)
encoder_LSTM_var/saved_output:0,(20, 128)
encoder_LSTM_var/saved_state:0,(20, 128)


In [ ]:
tf.get_collection("loss")

In [ ]:
tf.concat(0, tf.get_collection("outputs")[0])

In [ ]:
tf.get_collection("loss")

In [ ]:
tf.matmul(tf.concat(0, tf.get_collection("outputs")[0]), L.w)